In [1]:
import tensorflow as tf

/home/jin/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
class MLP(object):
    def __init__(self, X, n_hiddens, n_in, n_out):
        self.X = X
        for i, n_hidden in enumerate(n_hiddens):
            if i == 0:
                input = X
                input_dim = n_in
            else:
                input = output
                input_dim = n_hiddens[i-1]

            layer_name = "layer_"+str(i+1)

            with tf.variable_scope(layer_name, reuse=tf.AUTO_REUSE):
                W = self._weight_variable(layer_name, [input_dim, n_hidden])
                b = self._bias_variable(layer_name, [n_hidden])

                output = tf.nn.softmax_cross_entropy_with_logits_v2(tf.matmul(input, W) + b)

        layer_name = "output"
        with tf.variable_scope(layer_name, reuse=tf.AUTO_REUSE):
            W = self._weight_variable(layer_name, [n_hiddens[-1], n_out])
            b = self._bias_variable(layer_name, [n_out])
            self.hypothesis = tf.matmul(output, W) + b

    def _weight_variable(self, layer_name, shape):
        return tf.get_variable(
            layer_name + "_w", shape=shape,
            initializer=tf.random_normal_initializer)

    def _bias_variable(self, layer_name, shape):
        return tf.get_variable(
            layer_name + "_bias", shape=shape,
            initializer=tf.random_normal_initializer)

In [7]:
import tensorflow as tf
import numpy as np
import os

In [9]:
import tensorflow as tf
import numpy as np
import os

def loss_function(hypothesis, Y):
    with tf.variable_scope("loss", reuse=tf.AUTO_REUSE):
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(
                logits=hypothesis, labels=Y), name="softmax_cross_entropy")
    return loss


def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)
    return train_step


def decode(serialized_example):
    features = tf.parse_single_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.string),
      })


    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, [1024,]) 
    #image.set_shape((784))
    image = tf.cast(image, tf.float32)

    label = tf.decode_raw(features['label'], tf.uint8)
    label.set_shape((120))
    label = tf.cast(label, tf.float32)

    return image, label


def train(config_dict):
    X = tf.placeholder(tf.float32, [None, 32*32])
    Y = tf.placeholder(tf.float32, [None, 120])

    n_in = 32*32
    n_hiddens = config_dict["n_hiddens"]
    n_out = 120

    mlp_model = MLP(X, n_hiddens=n_hiddens, n_in=n_in, n_out=n_out)
    loss = loss_function(mlp_model.hypothesis, Y)
    train_step = training(loss, learning_rate=config_dict["learning_rate"])

    import glob
    filenames = glob.glob("./tfrecords/general/train/*.tfrecords")
    train_dataset = tf.data.TFRecordDataset(filenames)
    train_dataset = train_dataset.map(decode)
    train_dataset = train_dataset.shuffle(buffer_size=10000)
    train_dataset = train_dataset.batch(config_dict["batch_size"])
    iterator = tf.data.Iterator.from_structure(
                            train_dataset.output_types,
                            train_dataset.output_shapes)
    next_element = iterator.get_next()
    train_init_op = iterator.make_initializer(train_dataset)

    filenames = glob.glob("./tfrecords/general/test/*.tfrecords")
    test_dataset = tf.data.TFRecordDataset(filenames)
    test_dataset = test_dataset.map(decode)
    test_dataset = test_dataset.batch(105824)
    test_iterator = tf.data.Iterator.from_structure(
                            test_dataset.output_types,
                            test_dataset.output_shapes)
    test_next_element = test_iterator.get_next()
    test_init_op = test_iterator.make_initializer(test_dataset)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for epoch in range(config_dict["training_epochs"]):
            # initialize the iterator on the training data
            sess.run(train_init_op)

            # get each element of the training dataset until the end is reached
            cost = 0
            total_batch = 0
            while True:
                try:
                    batch = sess.run(next_element)
                    batch_xs = batch[0]
                    batch_ys = batch[1]

                    feed_dict = {X: batch_xs, Y: batch_ys}
                    c, _ = sess.run([loss, train_step], feed_dict=feed_dict)
                    cost += c
                    total_batch += 1

                except tf.errors.OutOfRangeError:
                    print("End of training dataset.")
                    break

            avg_cost = cost / total_batch
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(
                    avg_cost))

            sess.run(test_init_op)
            while True:
                try:
                    batch = sess.run(test_next_element)
                    batch_x = batch[0]
                    batch_y = batch[1]

                    correct_prediction = tf.equal(
                            tf.argmax(mlp_model.hypothesis, axis=1),
                            tf.argmax(Y, axis=1))
                    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                    print('Accuracy:', sess.run(accuracy, feed_dict={
                          X: batch_x, Y: batch_y}))
                except tf.errors.OutOfRangeError:
                    break


if __name__ == "__main__":
    config_dict = {
        "learning_rate": 0.01,
        "n_hiddens": [1024, 512, 256],
        "training_epochs": 300,
        "batch_size": 256
        }

    train(config_dict)

End of training dataset.
Epoch: 0001 cost = 5.022602015
Accuracy: 0.0072886297
End of training dataset.
Epoch: 0002 cost = 4.805653594
Accuracy: 0.011418853
End of training dataset.
Epoch: 0003 cost = 4.784485766
Accuracy: 0.011418853
End of training dataset.
Epoch: 0004 cost = 4.782761024
Accuracy: 0.011418853
End of training dataset.
Epoch: 0005 cost = 4.783287643
Accuracy: 0.011418853
End of training dataset.
Epoch: 0006 cost = 4.783010886
Accuracy: 0.011418853
End of training dataset.
Epoch: 0007 cost = 4.783306518
Accuracy: 0.011418853
End of training dataset.
Epoch: 0008 cost = 4.783097311
Accuracy: 0.011418853
End of training dataset.
Epoch: 0009 cost = 4.782759549
Accuracy: 0.011418853
End of training dataset.
Epoch: 0010 cost = 4.783156483
Accuracy: 0.011418853
End of training dataset.
Epoch: 0011 cost = 4.783147020
Accuracy: 0.011418853
End of training dataset.
Epoch: 0012 cost = 4.783402641
Accuracy: 0.011418853
End of training dataset.
Epoch: 0013 cost = 4.783111792
Accurac

Accuracy: 0.011418853
End of training dataset.
Epoch: 0107 cost = 4.782533257
Accuracy: 0.011418853
End of training dataset.
Epoch: 0108 cost = 4.782216762
Accuracy: 0.011418853
End of training dataset.
Epoch: 0109 cost = 4.782872097
Accuracy: 0.011418853
End of training dataset.
Epoch: 0110 cost = 4.782583427
Accuracy: 0.011418853
End of training dataset.
Epoch: 0111 cost = 4.782785027
Accuracy: 0.011418853
End of training dataset.
Epoch: 0112 cost = 4.782442951
Accuracy: 0.011418853
End of training dataset.
Epoch: 0113 cost = 4.782595246
Accuracy: 0.011418853
End of training dataset.
Epoch: 0114 cost = 4.782375196
Accuracy: 0.011418853
End of training dataset.
Epoch: 0115 cost = 4.782237897
Accuracy: 0.011418853
End of training dataset.
Epoch: 0116 cost = 4.782054974
Accuracy: 0.011418853
End of training dataset.
Epoch: 0117 cost = 4.782218669
Accuracy: 0.011418853
End of training dataset.
Epoch: 0118 cost = 4.782403476
Accuracy: 0.011418853
End of training dataset.
Epoch: 0119 cost 

End of training dataset.
Epoch: 0212 cost = 4.782663331
Accuracy: 0.011418853
End of training dataset.
Epoch: 0213 cost = 4.782078112
Accuracy: 0.011418853
End of training dataset.
Epoch: 0214 cost = 4.782346997
Accuracy: 0.011418853
End of training dataset.
Epoch: 0215 cost = 4.782367340
Accuracy: 0.011418853
End of training dataset.
Epoch: 0216 cost = 4.782904940
Accuracy: 0.011418853
End of training dataset.
Epoch: 0217 cost = 4.782840347
Accuracy: 0.011418853
End of training dataset.
Epoch: 0218 cost = 4.782336470
Accuracy: 0.011418853
End of training dataset.
Epoch: 0219 cost = 4.781974770
Accuracy: 0.011418853
End of training dataset.
Epoch: 0220 cost = 4.782551773
Accuracy: 0.011418853
End of training dataset.
Epoch: 0221 cost = 4.782635887
Accuracy: 0.011418853
End of training dataset.
Epoch: 0222 cost = 4.782428734
Accuracy: 0.011418853
End of training dataset.
Epoch: 0223 cost = 4.782420056
Accuracy: 0.011418853
End of training dataset.
Epoch: 0224 cost = 4.782903987
Accuracy